In [1]:
"""Portfolios and Portfolio Groups

Demonstrates how to do various operations with portfolios and portfolio groups.
Attributes
----------
transactions
portfolios
portfolio groups
corporate actions
commands
"""

'Portfolios and Portfolio Groups\n\nDemonstrates how to do various operations with portfolios and portfolio groups.\nAttributes\n----------\ntransactions\nportfolios\nportfolio groups\ncorporate actions\ncommands\n'

## Portfolios and Portfolio Groups

Portfolios in LUSID are used to contain transactions, and come in multiple forms based on hierarchy and use case. The basic portfolio types are :
- Transaction Portfolios
- Reference Portfolios
- Derived Portfolios

[LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2)

These portfolios can then be grouped, where portfolio operations on the group are applied to an aggregated set of portfolios in the hierarchy. 

[LUSID Knowledge Base: Portfolio Groups](https://support.lusid.com/how-do-you-group-and-aggregate-portfolios)

This example will show how to :
1. Create Transactions Portfolios
2. Create Derived Portfolios
3. Portfolio Details
    - 3.1. Get Details
    - 3.2. Add/Modify Details
    - 3.3. Delete Details
4. Create Portfolio Groups
5. Add Portfolios to a Group
6. Get Portfolio Group Details
7. Add a Group to a Group (Creation of Subgroups)
8. Modify Groups
9. Get Full Expansion of Group
10. Remove Portfolios and Subgroups from Groups
11. Delete Groups
12. Get Group Commands
13. List Groups in Scope
14. List Portfolios in Scope
15. Delete Portfolios

Initialise our environment and connect to LUSID

*Run the cell below to initialise the environment*

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Environment Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.6.4789.0


First we start by loading some default portfolios, we will put them in two different scopes to demonstrate the separation between scopes and the usage of different scopes within portfolio groups.

- Note: In order to demonstrate the deletion more clearly at then end of this example, run the portfolio creation more than once, and the derived portfolio creation 2-3 times as well, this will create a population of unnecessary portfolios that can be safely removed without affecting the portfolios of interest. It is not necessary to do this, the deletion example still works with a single set of portfolios

### Initial Setup
*Run the cell below to load a csv of portfolio names*

In [3]:
# Use pandas to read a csv containing the information needed to create some portfolios
df = pd.read_csv("data/dummy_portfolios_names.csv")
df.head()

,display_name,base_currency
0,portfolio-A,GBP
1,portfolio-B,GBP
2,portfolio-C,GBP
3,portfolio-D,GBP
4,portfolio-E,GBP


In [4]:
# Create Two Scopes
scope1 = f'portfolio_demo_{create_scope_id()}'
scope2 = f'portfolio_demo_xyz_{create_scope_id()}'
print("Scope1 : ", scope1)
print("Scope2 : ", scope2)

Scope1 :  portfolio_demo_38a5-0a6e-84e1-90
Scope2 :  portfolio_demo_xyz_38a5-0a6e-84e1-90


The cell below contains a number of helper functions used throughout this notebook - these are not necessary for any of the methods demonstrated, but simplify the readability and usability of this module

In [5]:
# The following function extracts resource ids from a list of portfolios
def get_resource_ids_from_response(responses):
    resource_id_list = []
    for portfolio in responses:
        resource_id_list.append(portfolio.id)
    return resource_id_list

## 1. Create Transactions Portfolios
We begin by upserting the csv data into LUSID in the form of new transaction portfolios

*Run the cell below to create portfolios and print their information*

In [6]:
# Create our portfolios from the loaded CSV data
effective_date = datetime(2018, 1, 1, tzinfo=pytz.utc).isoformat()
created_portfolios = []
for index, row in df.iterrows():
    print("Creating portfolio ", index)
    # Select the scope for each portfolio based on currency, for demonstration purposes we will
    # split GBP based portfolios into scope 1 and USD based portfolios into scope 2
    if row['base_currency'] == 'USD':
        portfolio_scope = scope2
    else:
        portfolio_scope = scope1
    # Next we create a request for the portfolio
    request = models.CreateTransactionPortfolioRequest(
        code=row['display_name'],
        display_name=row['display_name'],
        base_currency=row['base_currency'],
        created=effective_date,
        description=None,
        corporate_action_source_id=None,
        accounting_method=None,
        sub_holding_keys=None,
        properties=None)
    # And finally we can upsert the portfolio creation request to LUSID
    result = api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
        scope=portfolio_scope,
        create_transaction_portfolio_request=request)
    # Save the portfolio to a list for easy access later on
    created_portfolios.append(result)
    print('...')

# we can now prettyprint all our portfolio responses to see the info on each new portfolio
for portfolio in created_portfolios:
    prettyprint.portfolio_response(portfolio)

Creating portfolio  0
...
Creating portfolio  1
...
Creating portfolio  2
...
Creating portfolio  3
...
Creating portfolio  4
...
Creating portfolio  5
...
Creating portfolio  6
...
Creating portfolio  7
...
Portfolio Created
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:11.530048+00:00

Portfolio Created
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:12.270559+00:00

Portfolio Created
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:13.138666+00:00

Portfolio Created
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
Portfolio Effect

## 2. Create Derived Portfolio
We then create one derived portfolio from the first transaction portfolio created.

[LUSID Knowledge Base: Derived Portfolio](https://support.lusid.com/what-is-a-derived-portfolio-and-how-are-they-used) 

*Run the cell below to create a derived portfolio and print its information*

In [7]:
# Create derived transaction portfolio from the first portfolio in the list

derived_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
    # this uses the portfolio name
    display_name="derived-"+created_portfolios[0].display_name,
    code="derived-"+created_portfolios[0].display_name,
    parent_portfolio_id=models.ResourceId(
        scope=created_portfolios[0].id.scope,
        code=created_portfolios[0].id.code),
    description="derived_portfolio_description",
    created=created_portfolios[0].created,
    corporate_action_source_id=None,
    accounting_method=None,
    sub_holding_keys=None)

derived_portfolio = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi).create_derived_portfolio(
    scope=scope1,
    create_derived_transaction_portfolio_request=derived_portfolio_request)

prettyprint.portfolio_response(derived_portfolio)

Derived Portfolio Created
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:18.294584+00:00

   Parent Portfolio Details
   Scope: portfolio_demo_38a5-0a6e-84e1-90
   Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08



## 3. Portfolio Details
#### 3.1 Get Details
A derived portfolio will automatically inherit all of the parent portfolio's details if they are not specified. These details can be modified and/or deleted. 
Modifying the details on a derived portfolio will not affect the parent portfolio. This is demonstrated in the following cells

*Run the cell below to get details on the derived portfolio*

In [8]:
# First we will get the details for our new derived portfolio
# and check they match with the parent portfolio
derived_details = api_factory.build(lusid.api.TransactionPortfoliosApi).get_details(
    scope=scope1,
    code=derived_portfolio.id.code)
print("Derived Portfolio Details : ")
prettyprint.portfolio_details_response(derived_details)

print("Parent Portfolio Details : ")
parent_details = api_factory.build(lusid.api.TransactionPortfoliosApi).get_details(
    scope=scope1,
    code=created_portfolios[0].id.code)
prettyprint.portfolio_details_response(parent_details)

Derived Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_38a5-0a6e-84e1-90
Detail Origin Portfolio Code: derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
Base Currency: GBP
Corporate Action Source Id: None


Parent Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_38a5-0a6e-84e1-90
Detail Origin Portfolio Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
Base Currency: GBP
Corporate Action Source Id: None




#### 3.2 Add/Modify Details
Now we can modify the derived portfolio details by adding a Corporate Action Source so it no longer matches the parent portfolio

*Run the cell below to modify the details of the derived portfolio*

In [9]:
response = api_factory.build(lusid.api.CorporateActionSourcesApi).create_corporate_action_source(
    create_corporate_action_source_request=models.CreateCorporateActionSourceRequest(
        scope=scope1,
        code=derived_portfolio.id.code,
        display_name=f"Corporate Actions for {derived_portfolio.id.code}")
)

request = models.CreatePortfolioDetails(
    corporate_action_source_id=models.ResourceId(
        scope=scope1,
        code=derived_portfolio.id.code)
)

response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_portfolio_details(
    scope=scope1,
    code=derived_portfolio.id.code,
    create_portfolio_details=request)
print("Derived Portfolio Details : ")
prettyprint.portfolio_details_response(response)

print("Parent Portfolio Details : ")
parent_details = api_factory.build(lusid.api.TransactionPortfoliosApi).get_details(
    scope=scope1,
    code=created_portfolios[0].id.code)
prettyprint.portfolio_details_response(parent_details)

Derived Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_38a5-0a6e-84e1-90
Detail Origin Portfolio Code: derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
Base Currency: GBP
Corporate Action Source Id: {'code': 'derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec',
 'scope': 'portfolio_demo_38a5-0a6e-84e1-90'}


Parent Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_38a5-0a6e-84e1-90
Detail Origin Portfolio Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
Base Currency: GBP
Corporate Action Source Id: None




#### 3.3 Delete Details
Now we can delete portfolio details. Note how after deleting the details on the derived portfolio, the response returns the scope and code of the parent portfolio, since `get_details` only returns the id of the portfolio from which the details originate - not the portfolio being queried.

*Run the cell below to delete the details of the derived portfolio*

In [10]:
response = api_factory.build(lusid.api.DerivedTransactionPortfoliosApi).delete_derived_portfolio_details(
    scope=scope1,
    code=derived_portfolio.id.code
)

derived_details = api_factory.build(lusid.api.TransactionPortfoliosApi).get_details(
    scope=scope1,
    code=derived_portfolio.id.code)

print("Derived Portfolio Details : ")
prettyprint.portfolio_details_response(derived_details)

print("Parent Portfolio Details : ")
parent_details = api_factory.build(lusid.api.TransactionPortfoliosApi).get_details(
    scope=scope1,
    code=created_portfolios[0].id.code)
prettyprint.portfolio_details_response(parent_details)

Derived Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_38a5-0a6e-84e1-90
Detail Origin Portfolio Code: derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
Base Currency: GBP
Corporate Action Source Id: None


Parent Portfolio Details : 
Portfolio Details: 
Detail Origin Portfolio Scope: portfolio_demo_38a5-0a6e-84e1-90
Detail Origin Portfolio Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
Base Currency: GBP
Corporate Action Source Id: None




## 4. Create Portfolio Group
Next we can create some portfolio groups. We will then demonstrate the process of adding portfolios to a group, creating subgroups, and modifying group information.

To begin with, let's create a group in our main scope that contains all the GBP parent portfolios.

Note that the second group will be in a different scope to the first, it is entirely possible to combine portfolios and groups in different scopes.

*Run the cell below to create a pre-populate portfolio group*

In [11]:
# Create a portfolio group -- create two
# -- The first group we will create with all the portfolios in the first scope
# -- we can use a helper function to obtain a list of resource ids for this
scope_1_portfolios = [(x) for x in created_portfolios if x.id.scope == scope1]
scope_1_resource_ids = get_resource_ids_from_response(scope_1_portfolios)

# Create group 1 with the parent portfolios of scope 1 in it

group_request1 = models.CreatePortfolioGroupRequest(
    code="portfolio_group_code_1",
    display_name="portfolio_group1_name",
    created=effective_date,
    values=scope_1_resource_ids,
    sub_groups=None,
    description=None)

portfolio_group1 = api_factory.build(lusid.api.PortfolioGroupsApi).create_portfolio_group(
    scope=scope1,
    create_portfolio_group_request=group_request1)

prettyprint.portfolio_group_response(portfolio_group1, 'created')

Portfolio Group Created
Name: portfolio_group1_name
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572
portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f




## 5. Add Portfolio To Group
So we have one group pre-populated with the portfolios in the first scope `portfolio_demo_abc`. Since the derived portfolio hasn't been included in the first group, we can add it in now to Group 1.

*Run the cell below to add the derived portfolio to group 1*

In [12]:
# adding the derived portfolio to group 1 :
add_response = api_factory.build(lusid.api.PortfolioGroupsApi).add_portfolio_to_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    resource_id=derived_portfolio.id,
    effective_at=portfolio_group1.created)

prettyprint.get_portfolio_group_response(add_response)

Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572
portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
Subgroups Inside Group: 




## 6. Get Portfolio Group Details
We can now compare two methods of obtaining groups from LUSID. The first is `get portfolio group` while the second is `get portfolio group expansion`. The difference here is that the first method will return only information about the specified group, including the ids of the portfolios and subgroups it contains, but not any further details of its portfolios and subgroups. The second method will not only return this basic information, but also the full details of each portfolio in the group and each portfolio in every subgroup as well. 

First we compare group 1 at the time of creation and at the most up-to-date time below, so we can see the changes that have been made to it since we created it using `get portfolio group`. The only expected difference right now will be the added derived portfolio. We will demonstrate the expanded method further on.

*Run the cell below to compare the group info now and at time of creation*

In [13]:
# Get portfolio group now
updated_group1 = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code)

# Get portfolio group 1 prior to the update
original_group1 = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    as_at=portfolio_group1.version.as_at_date)

print('Group 1 at time of creation')
prettyprint.get_portfolio_group_response(original_group1)
print('')
print('Group 1 at latest time')
prettyprint.get_portfolio_group_response(updated_group1)

Group 1 at time of creation
Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572
portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
Subgroups Inside Group: 



Group 1 at latest time
Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
portfolio-E-eedd3eed-015e-4071-b

This leaves the two portfolios in Scope 2. Since they are in another scope and based in a different currency, it makes sense to place them in a separate group to the first one (although they could also be added to the first groups regardless of scope being different). This time we create an empty group, then populate it separately. We follow this process as opposed to creating a pre-populated group (such as group 1 above) to demonstrate how empty groups can be created and how to upsert multiple portfolios to a group.

*Run the cell below to create a second group containing no portfolios, and subsequently add portfolios in a separate command*

In [14]:
# Create group 2 with no portfolios

group_request2 = models.CreatePortfolioGroupRequest(
    code="portfolio_group_code_2",
    display_name="portfolio_group2_name",
    values=None,
    sub_groups=None,
    description=None)

portfolio_group2 = api_factory.build(lusid.api.PortfolioGroupsApi).create_portfolio_group(
    scope=scope2, 
    create_portfolio_group_request=group_request2)

# Print the newly created group info
prettyprint.portfolio_group_response(portfolio_group2, 'created')

# Add portfolios to groups
# Once again we get a list of resource ids for the portfolios in scope 2 this time
scope_2_portfolios = [(x) for x in created_portfolios if x.id.scope == scope2]
scope_2_resource_ids = get_resource_ids_from_response(scope_2_portfolios)

# Then to add multiple portfolios to group 2, we have to iterate over the list of ids, since they have to
# be added one by one.
for resource_id in scope_2_resource_ids:
    api_factory.build(lusid.api.PortfolioGroupsApi).add_portfolio_to_group(
        scope=portfolio_group2.id.scope,
        code=portfolio_group2.id.code,
        resource_id=resource_id,
        effective_at=portfolio_group2.created)

populated_group2 = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code)

prettyprint.get_portfolio_group_response(populated_group2)

Portfolio Group Created
Name: portfolio_group2_name
Scope: portfolio_demo_xyz_38a5-0a6e-84e1-90
Code: portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f
Portfolios Inside Group: 


Portfolio Group: 
Name: portfolio_group2_name
Scope: portfolio_demo_xyz_38a5-0a6e-84e1-90
Code: portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f
Portfolios Inside Group: 
portfolio-Y-0e12296d-722d-41a4-ae80-6f906d9f02f9
portfolio-X-6af65d97-8e74-4a3c-94ad-414fc7296d93
Subgroups Inside Group: 




## 7. Add Group To Group (Create Subgroup)
We can also combine groups into each other to create sub-groups. We will add group 2 as a subgroup in group 1.

*Run the cell below to add Group 2 as a subgroup to Group 1*

In [15]:
# Add group to group
add_response = api_factory.build(lusid.api.PortfolioGroupsApi).add_sub_group_to_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    resource_id=portfolio_group2.id,
    effective_at=portfolio_group2.created)

prettyprint.get_portfolio_group_response(add_response)

Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572
portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
Subgroups Inside Group: 
portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f




## 8. Modify Group
Since group 2 is now a subgroup, we can update its display name to reflect this

*Run the cell below to modify the group name*

In [16]:
# Update group request
group_update_request = models.UpdatePortfolioGroupRequest(
    display_name="Group1_Subgroup",
    description=None)

# Update group
update_response = api_factory.build(lusid.api.PortfolioGroupsApi).update_portfolio_group(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code,
    update_portfolio_group_request=group_update_request,
    effective_at=portfolio_group2.created)

prettyprint.portfolio_group_response(update_response, 'Updated')

Portfolio Group
Name: Group1_Subgroup
Scope: portfolio_demo_xyz_38a5-0a6e-84e1-90
Code: portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f
Portfolios Inside Group: 
portfolio-Y-0e12296d-722d-41a4-ae80-6f906d9f02f9
portfolio-X-6af65d97-8e74-4a3c-94ad-414fc7296d93




## 9. Get Full Expansion of a Group
We can now compare the `get _portfolio_group` method with getting the full expansion of a group. You will see that on top of a list of portfolios and subgroups, the full expansion also returns all the information for the portfolios similar to running the `get_portfolio` method on each portfolio in the group and groubgroup.

*Run the cell below to get the full expansion of Group 1*

In [17]:
# Get the basic group information for group 1 again:
updated_group1 = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code)

print('Group 1 at latest time with basic GET method : ')
prettyprint.get_portfolio_group_response(updated_group1)

# Get full expansion of portfolio group 1:
expansion = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group_expansion(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code)
print('Group 1 at latest time with full expansion method : ')
prettyprint.expanded_portfolio_group_response(expansion)

Group 1 at latest time with basic GET method : 
Portfolio Group: 
Name: portfolio_group1_name
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572
portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
Subgroups Inside Group: 
portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f


Group 1 at latest time with full expansion method : 
Portfolio Group Full Details : 
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
Portfolio 
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
Portfolio Effective From: 2018-01-

## 10. Remove Portfolios and Subgroups from Group
In LUSID it is possible to delete a populated group without deleting the portfolios, they will simply be removed from the group hierarchy and will remain ungrouped in their respective scopes. Our Subgroup contains two portfolios, so for demonstration purposes we are carrying out each step individually of removing the portfolios from the group, removing the subgroup from the main group, and then deleting it as it is no longer needed.

*Run the cell below to remove portfolios from Group 2*

In [18]:
# 1. Removal of portfolios from a group has to be done one portfolio at a time (deleting the group will 
# remove all of them in one go as mentioned in the cell above), thus we can loop through the list of 
# portfolios in scope 2 since they are all contained in the subgroup

# first we print out the details of group 2
original_group2 = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code)

prettyprint.get_portfolio_group_response(original_group2)
print('')

# then we proceed with removing each portfolio in the group, you will see each printed response will have
# one fewer portfolio ids attached to the group.
for portfolio_id in scope_2_resource_ids:
    portfolio_removal = api_factory.build(lusid.api.PortfolioGroupsApi).delete_portfolio_from_group(
        scope=portfolio_group2.id.scope,
        code=portfolio_group2.id.code,
        portfolio_scope=portfolio_id.scope,
        portfolio_code=portfolio_id.code,
        effective_at=portfolio_group2.created)
    prettyprint.portfolio_group_response(portfolio_removal, 'none')

Portfolio Group: 
Name: Group1_Subgroup
Scope: portfolio_demo_xyz_38a5-0a6e-84e1-90
Code: portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f
Portfolios Inside Group: 
portfolio-Y-0e12296d-722d-41a4-ae80-6f906d9f02f9
portfolio-X-6af65d97-8e74-4a3c-94ad-414fc7296d93
Subgroups Inside Group: 



Portfolio Group
Name: Group1_Subgroup
Scope: portfolio_demo_xyz_38a5-0a6e-84e1-90
Code: portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f
Portfolios Inside Group: 
portfolio-Y-0e12296d-722d-41a4-ae80-6f906d9f02f9


Portfolio Group
Name: Group1_Subgroup
Scope: portfolio_demo_xyz_38a5-0a6e-84e1-90
Code: portfolio_group-fd5076f3-8e6b-4e00-b5f5-7eff09d99e9f
Portfolios Inside Group: 




*Run the cell below to remove Group 2 from Group 1*

In [19]:
# 2. Remove group from group
group_removal = api_factory.build(lusid.api.PortfolioGroupsApi).delete_sub_group_from_group(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code,
    subgroup_scope=portfolio_group2.id.scope,
    subgroup_code=portfolio_group2.id.code,
    effective_at=portfolio_group1.created)

prettyprint.portfolio_group_response(group_removal, 'none')
print('Remaining Sub Groups : ', group_removal.sub_groups)

Portfolio Group
Name: portfolio_group1_name
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Portfolios Inside Group: 
derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572
portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19


Remaining Sub Groups :  []


## 11. Delete Group
*Run the cell below to delete group 2*

In [20]:
# 3. Delete group
group_deletion = api_factory.build(lusid.api.PortfolioGroupsApi).delete_portfolio_group(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code)

## 12. Get Group Commands
We can now check what operations have been applied to each of our groups, including an already deleted group, by using the `get portfolio group commands` method, as below.

*Run the cell below to get all commands from groups 1 and 2*

In [21]:
# Get commands
group1_commands = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group_commands(
    scope=portfolio_group1.id.scope,
    code=portfolio_group1.id.code)

group2_commands = api_factory.build(lusid.api.PortfolioGroupsApi).get_portfolio_group_commands(
    scope=portfolio_group2.id.scope,
    code=portfolio_group2.id.code)

prettyprint.group_commands(group1_commands, portfolio_group1.display_name)

prettyprint.group_commands(group2_commands, portfolio_group2.display_name)

Commands Applied To Group portfolio_group1_name
Number of commands : 2
Description : Create portfolio group
At Time : 2020-07-10 18:38:22.597270+00:00


Description : Add portfolio to group
At Time : 2020-07-10 18:38:23.206472+00:00


Description : Add subgroup to group
At Time : 2020-07-10 18:38:27.258429+00:00


Description : Delete subgroup from group
At Time : 2020-07-10 18:38:33.427782+00:00


Commands Applied To Group portfolio_group2_name
Number of commands : 2
Description : Create portfolio group
At Time : 2020-07-10 18:38:24.026617+00:00


Description : Add portfolio to group
At Time : 2020-07-10 18:38:24.686701+00:00


Description : Add portfolio to group
At Time : 2020-07-10 18:38:25.274428+00:00


Description : Update portfolio group
At Time : 2020-07-10 18:38:27.953089+00:00


Description : Delete portfolio from group
At Time : 2020-07-10 18:38:31.926426+00:00


Description : Delete portfolio from group
At Time : 2020-07-10 18:38:32.649497+00:00


Description : Delete port

## 13. List Groups in Scope
We can now verify that the group has indeed been deleted from our scope by getting all groups in each of our scopes, the second of which should be empty now.

Note : Scope 1 may have groups remaining from previous runs of this notebook example

*Run the cell below to list all groups in scopes 1 and 2*

In [22]:
# List groups in scope - show one scope with no groups
scope1_groups = api_factory.build(lusid.api.PortfolioGroupsApi).list_portfolio_groups(
    scope=scope1)

scope2_groups = api_factory.build(lusid.api.PortfolioGroupsApi).list_portfolio_groups(
    scope=scope2)

prettyprint.groups_in_scope(scope1_groups)
prettyprint.groups_in_scope(scope2_groups)

Groups in Scope: 
Scope : portfolio_demo_38a5-0a6e-84e1-90
Code : portfolio_group-c0bd3b5f-20cf-4254-8fab-101c1aca7a48
Groups in Scope: 


In case any extra groups are hanging around, the next cell will remove all groups. This will also remove Group 1 in Scope 1, which is still populated, showing how the portfolios remain in the scope and ungrouped following this operation.

*Run the cell below to remove any remaining groups in both scopes*

In [23]:
# create a list of all groups in both scopes
groups_to_delete = []
for group in scope1_groups.values:
    groups_to_delete.append(group.id)
for group in scope2_groups.values:
    groups_to_delete.append(group.id)

# loop through the list of groups and delete each one, one by one
for group in groups_to_delete:
    api_factory.build(lusid.api.PortfolioGroupsApi).delete_portfolio_group(
        scope=group.scope,
        code=group.code)

# List groups in scope - both will now be empty
scope1_groups = api_factory.build(lusid.api.PortfolioGroupsApi).list_portfolio_groups(
    scope=scope1)

scope2_groups = api_factory.build(lusid.api.PortfolioGroupsApi).list_portfolio_groups(
    scope=scope2)

prettyprint.groups_in_scope(scope1_groups)
prettyprint.groups_in_scope(scope2_groups)

Groups in Scope: 
Groups in Scope: 


## 14. List Portfolios in Scope
If we list all the portfolios in our scopes we will see that they are all still there. There may be extra portfolios not created in this example, if this is the case, we can delete all these extra portfolios and keep only those we are interested in. 

*Run the cell below to list all the portfolios in both scopes*

In [24]:
scope1_portfolios_list_response = api_factory.build(lusid.api.PortfoliosApi).list_portfolios_for_scope(
    scope=scope1)

scope2_portfolios_list_response = api_factory.build(lusid.api.PortfoliosApi).list_portfolios_for_scope(
    scope=scope2)

print('Portfolios remaining in scope ', scope1, ' :\n')
for portfolio in scope1_portfolios_list_response.values:
    prettyprint.expansion_portfolio_response(portfolio)

print('Portfolios remaining in scope ', scope2, ' :\n')
for portfolio in scope2_portfolios_list_response.values:
    prettyprint.expansion_portfolio_response(portfolio)

Portfolios remaining in scope  portfolio_demo_38a5-0a6e-84e1-90  :

Derived Portfolio 
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:35.845334+00:00

   Parent Portfolio Details
   Scope: portfolio_demo_38a5-0a6e-84e1-90
   Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08

Portfolio 
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:35.845334+00:00

Portfolio 
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:35.845334+00:00

Portfolio 
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
Portfolio Effective From: 2018-01-01 0

## 15. Delete Portfolios
The cells below allow us to delete the extra portfolios in the scope. Keep in mind, derived portfolios must be deleted first, as if a parent portfolio still contains a child, the deletion will fail.

*Run the cell below to create some helper functions*

In [25]:
# Helper functions to filter out unwanted portfolios
def extract_ids(portfolioList):
    parent_ids = []
    derived_ids = []
    for folio in portfolioList:
        if folio.is_derived:
            derived_ids.append((folio.id.scope, folio.id.code))
        else:
            parent_ids.append((folio.id.scope, folio.id.code))
    return (parent_ids, derived_ids)


def return_not_matches(a, b):
    if type(b) is not tuple:
        return [x for x in a if x not in b]
    else:
        return [x for x in a if x not in [b]]

Now we select the parent/derived portfolios we wish to keep or delete. Currently `portfolios_to_keep` is hardcoded to keep the derived portfolio that we created at the beginning of this example, and its parent portfolio.

*Run the cell below to filter and separate portfolios to keep and delete in each scope*

In [26]:
# first combine all the portfolios in our two scopes
all_portfolios_in_scopes = extract_ids(scope1_portfolios_list_response.values
                                       + scope2_portfolios_list_response.values)
parent_portfolios = all_portfolios_in_scopes[0]
derived_portfolios = all_portfolios_in_scopes[1]

# then combine the portfolios we're interested in keeping into a list
# - just the first portfolio we created and its derived portfolio
parents_to_keep = (derived_portfolio.parent_portfolio_id.scope, derived_portfolio.parent_portfolio_id.code)
derived_to_keep = (derived_portfolio.id.scope, derived_portfolio.id.code)

# we can now exclude portfolios to keep from the full list
parents_to_delete = return_not_matches(parent_portfolios, parents_to_keep)
derived_to_delete = return_not_matches(derived_portfolios, derived_to_keep)

prettyprint.portfolio_filtering(
    parents_to_keep, derived_to_keep, parents_to_delete, derived_to_delete)

Keeping : 
    Parents: 
    Scope : portfolio_demo_38a5-0a6e-84e1-90
    Code : portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08

    Derived: 
    Scope : portfolio_demo_38a5-0a6e-84e1-90
    Code : derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec

Deleting : 
    Parents: 
    Scope : portfolio_demo_38a5-0a6e-84e1-90
    Code : portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19

    Scope : portfolio_demo_38a5-0a6e-84e1-90
    Code : portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f

    Scope : portfolio_demo_38a5-0a6e-84e1-90
    Code : portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb

    Scope : portfolio_demo_38a5-0a6e-84e1-90
    Code : portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572

    Scope : portfolio_demo_38a5-0a6e-84e1-90
    Code : portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210

    Scope : portfolio_demo_xyz_38a5-0a6e-84e1-90
    Code : portfolio-X-6af65d97-8e74-4a3c-94ad-414fc7296d93

    Scope : portfolio_demo_xyz_38a5-0a6e-84e1-90
    Code : portfolio-Y-0e1

With this filtering done, we can proceed with deleting unwanted portfolios. Deletion is done one portfolio at a time, therefore we must loop through the list of portfolios to delete. Derived portfolios are deleted first, followed by a separate loop to delete parent (top level - they may never have had derived portfolios attached) portfolios.

*Run the cell below to delete all portfolios that are not being kept*

In [27]:
# Next we delete the derived portfolios first
print("Deleting Derived Portfolios ... ")
for portfolio in derived_to_delete:
    response = api_factory.build(lusid.api.PortfoliosApi).delete_portfolio(
        scope=portfolio[0],
        code=portfolio[1])
    print('Portfolio deleted : ', portfolio[0], portfolio[1])
    print('At time : ', str(response.as_at))


print("Deleting remaining Portfolios ... ")
# Then we proceed with deleting the remaining parent portfolios
for portfolio in parents_to_delete:
    response = api_factory.build(lusid.api.PortfoliosApi).delete_portfolio(
        scope=portfolio[0],
        code=portfolio[1])
    print('Portfolio deleted : ', portfolio[0], portfolio[1])
    print('At time : ', str(response.as_at))

Deleting Derived Portfolios ... 
Deleting remaining Portfolios ... 
Portfolio deleted :  portfolio_demo_38a5-0a6e-84e1-90 portfolio-B-48505cdd-a70e-4335-98e8-9aa53beeda19
At time :  2020-07-10 18:38:38.962238+00:00
Portfolio deleted :  portfolio_demo_38a5-0a6e-84e1-90 portfolio-C-83ddedf3-dbe8-465a-a51f-f8092ae0bd7f
At time :  2020-07-10 18:38:40.022434+00:00
Portfolio deleted :  portfolio_demo_38a5-0a6e-84e1-90 portfolio-D-d8480b12-1891-49ea-82c3-f622fd652ebb
At time :  2020-07-10 18:38:40.994423+00:00
Portfolio deleted :  portfolio_demo_38a5-0a6e-84e1-90 portfolio-E-eedd3eed-015e-4071-b1e7-0db8d5c4c572
At time :  2020-07-10 18:38:41.894427+00:00
Portfolio deleted :  portfolio_demo_38a5-0a6e-84e1-90 portfolio-F-707ff0a2-c025-41e6-b8b3-c9b92f429210
At time :  2020-07-10 18:38:42.849627+00:00
Portfolio deleted :  portfolio_demo_xyz_38a5-0a6e-84e1-90 portfolio-X-6af65d97-8e74-4a3c-94ad-414fc7296d93
At time :  2020-07-10 18:38:43.916388+00:00
Portfolio deleted :  portfolio_demo_xyz_38a5-0

If we now re-list the portfolios in our scopes, we will see that only our two original portfolios remain, and one derived portfolio.  Note that scope 2 is now completely empty of portfolios.

*Run the cell below to list all portfolios in both scopes at the latest time*

In [28]:
# We can re-run the list portfolios command to check what portfolios remain in each scope
scope1_portfolios_list_response = api_factory.build(lusid.api.PortfoliosApi).list_portfolios_for_scope(
    scope=scope1)

scope2_portfolios_list_response = api_factory.build(lusid.api.PortfoliosApi).list_portfolios_for_scope(
    scope=scope2)

prettyprint.remaining_portfolios(scope1_portfolios_list_response, scope1)
prettyprint.remaining_portfolios(scope2_portfolios_list_response, scope2)

Portfolios remaining in scope: portfolio_demo_38a5-0a6e-84e1-90:
Derived Portfolio Created
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: derived-portfolio-A-35b88f95-22aa-41e0-9685-ee94b920f9ec
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:44.366425+00:00

   Parent Portfolio Details
   Scope: portfolio_demo_38a5-0a6e-84e1-90
   Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08

Portfolio Created
Scope: portfolio_demo_38a5-0a6e-84e1-90
Code: portfolio-A-0d361b51-ffb8-4e9c-98b9-97f4afe07e08
Portfolio Effective From: 2018-01-01 00:00:00+00:00
Portfolio Created On: 2020-07-10 18:38:44.366425+00:00

Portfolios remaining in scope: portfolio_demo_xyz_38a5-0a6e-84e1-90:
